In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
with open('dim_customer.sql') as f:
    lines = f.readlines()

Hundred characters

In [3]:
def hundred_chars(lines):
    print('Lines of SQL should be no longer than ~100 characters. Please use new line:')
    for count, i in enumerate(lines):
        cnt = 0
        for j in i:
            if j != ' ':
                cnt=cnt+1
        if cnt>100:
            print("\nLine no. {}\n{}".format(count+1,i.strip()))
    print('\n--------------------------------\n')

Comma at the end of the line

In [4]:
def comma_at_end(lines):
    lines_sign = []
    print("For following lines comma is present at start of the line: \n")
    for count, i in enumerate(lines):
        if i.lstrip() != '':  
            if i.lstrip()[0].__contains__(","):
                lines_sign.append([count,i])
                print('Line no. {}\n{}'.format(count,i.strip()))
    print('\n--------------------------------\n')

Inner Join instead of Join keyword 

In [5]:
def inner_join_keyword(lines):
    lines_join = []
    print('Inner keyword not present before join in following lines:\n')
    for count, i in enumerate(lines):
        if i.__contains__("join"):
            lines_join.append([count,i])

    lines_after_join = []
    flag = 0
    for count, i in enumerate(lines):
        if flag == 1:
            flag = 0
            lines_after_join.append([count, i])
        if i.__contains__("join"):
            flag = 1

    join_ls = ['inner', 'left', 'outer', 'right']
    for count, i in enumerate(lines_join):
        flag = 0
        for k in join_ls:
            if flag == 0:
                if k in i[1]:
                    flag = 1
        if flag ==0:
            print("Line no. {}\n{}".format(i[0]+1,i[1].strip()))
    print('\n--------------------------------\n')
    print("Use 'On' instead of 'Using' in following lines:\n")
    for count, i in enumerate(lines_after_join):
        if ' using' in i[1]:
            print(f"Line no. {i[0]+1}\n{i[1].strip()}")
    print('\n--------------------------------\n')

Not equals as !=

In [6]:
def not_equals_sign(lines):
    print('Use != instead of <> in following lines: \n')
    lines_sign = []
    for count, i in enumerate(lines):
        if i.__contains__("<>"):
            lines_sign.append([count,i])
    for count, i in enumerate(lines_sign):
        print("Line no. {}\n{}".format(i[0]+1,i[1].strip()))
    print('\n--------------------------------\n')

Check snake casing

In [7]:
def snake_casing(lines):
    lines_uppercase = []
    flag=0
    reg = '[A-Za-z]+.'
    print('Following lines contain uppercase characters:\n')
    for count, i in enumerate(lines):
        if i.islower()==False and i!='\n':
            x=i.split("'")
            for cnt,j in enumerate(x):
                if re.search(reg, j):
                    if cnt%2 ==0 and j.islower()==False and j!='\n':
                        flag=1
                        lines_uppercase.append([count+1,i])
                        print('Line no. {}:\n{}'.format(count+1,i.strip()))
    print('\n--------------------------------\n')

In [8]:
hundred_chars(lines)
comma_at_end(lines)
inner_join_keyword(lines)
not_equals_sign(lines)
snake_casing(lines)

Lines of SQL should be no longer than ~100 characters. Please use new line.

Line no. 70
qualify dense_rank() over ( partition by coalesce(salesforce_account_id, legacy_mindbody_sfdc_account_id, legacy_booker_sfdc_account_id)

Line no. 88
on dim_mindbody_legacy.legacy_mindbody_sfdc_account_id = core_software_subscription.legacy_mindbody_sfdc_account_id

Line no. 107
on dim_booker_legacy.legacy_booker_sfdc_account_id = core_software_subscription.legacy_booker_sfdc_account_id

Line no. 119
iff(core_software_subscription.platform='Mindbody', coalesce(to_varchar(dim_customer_mindbody_map.aria_id), dim_customer_mindbody_map.zuora_id), null)

Line no. 166
using dim_customer_mindbody_map.legacy_mindbody_sfdc_account_id = dim_mindbody_legacy.legacy_mindbody_sfdc_account_id

Line no. 186
on dim_customer_mindbody_map.legacy_mindbody_sfdc_account_id = mindbody_legacy.legacy_mindbody_sfdc_account_id

Line no. 193
{{dbt_utils.surrogate_key(['platform','coalesce(mindbody_studio_id::varchar,product_i